In [ ]:
!pip install -r requirements.txt -q

In [ ]:
from helpers.dataset import Conversation
from elasticsearch import Elasticsearch, helpers
from pydantic import BaseModel
import numpy as np
import logging
from typing import List
import os
from helpers.fair_forge import FairForge
import pandas as pd
import math
import re
from collections import defaultdict
from scipy.stats import spearmanr

In [ ]:
ELASTIC_URL = os.environ.get('ELASTIC_URL')
ELASTIC_AUTH = [os.environ.get('ELASTIC_AUTH_USER'), os.environ.get('ELASTIC_AUTH_PASSWORD')]
dataset = os.environ.get("dataset", "asb")
context_index = f"{dataset}-context"

In [ ]:
class ContextBatch(BaseModel):
    session_id: str
    context: str
    context_insight: str
    context_awareness: float
    context_thinkings: str
    qa_id: str
    assistant_id: str

class ContextMetric(BaseModel):
    session_id: str
    conv_thread: List[ContextBatch] = []

In [ ]:
def recreate_index(index_name: str, mapping: dict):
    if es.indices.exists(index=index_name):
        es.indices.delete(index=index_name)
        print(f"Index '{index_name}' deleted.")
    es.indices.create(index=index_name, body=mapping)
    print(f"Index '{index_name}' created.")

In [ ]:
es = Elasticsearch(
    ELASTIC_URL,
    basic_auth=tuple(ELASTIC_AUTH),
)

In [ ]:
class ContextAnalyzer(FairForge):
    def process(self, thread: Conversation):
        for batch in thread.conversation:
            query = batch.question
            # Get emotional distribution for ground truth and real assistant
            batch = ContextBatch(
                context="",
                context_insight="",
                context_awareness=0.0,
                context_thinkings="",
                session_id=thread.session_id,
                qa_id=batch.qa_id,
                assistant_id=thread.assistant_id
            )
            self.metrics.append(batch)

In [ ]:
context = ContextAnalyzer()
metrics = context.pipeline()

In [ ]:
mapping_context = {
  "mappings": {
    "properties": {
      "session_id": {"type": "keyword"},
      "context": {"type": "text"},
      "context_insight": {"type": "text"},
      "context_awareness": {"type": "float"},
      "context_thinkings": {"type": "text"},
      "assistant_id": {"type": "keyword"},
      "qa_id": {"type": "keyword"},
    }
  }
}